___

<a href='http://www.pieriandata.com'><img src='../Pierian_Data_Logo.png'/></a>
___
<center><em>Авторские права принадлежат Pierian Data Inc.</em></center>
<center><em>Для дополнительной информации посетите наш сайт <a href='http://www.pieriandata.com'>www.pieriandata.com</a></em></center>

# Проверочный проект по линейной регрессии

Мы изучили построение признаков, кросс-валидацию и поиск по сетке. Пришло время проверить Ваши новые навыки. Добро пожаловать на проверочный проект по машинному обучению! В этом проекте у нас будут достаточно подробные пошаговые инструкции. Будущие проекты будут уже в более свободной форме. Мы начнём проект с финальной версии набора данных Ames Housing, с которым мы работали в разделе "построение признаков" этого курса. Цель этого проекта - создать модель линейной регрессии, обучить её на данных с поиском оптимальных значений параметров по сетке, и затем оценить модель на тестовом наборе данных.

---
---
---
## Выполните задания, написанные жирным шрифтом

**ЗАДАНИЕ: Выполните ячейки ниже для импорта библиотек и загрузки данных. Возможно в будущем Вам понадобятся дополнительные команды import от scikit-learn.**

### Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Данные

In [4]:
df = pd.read_csv("../DATA/AMES_Final_DF.csv")

In [8]:
df.head(10)

,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,...,Sale Type_ConLw,Sale Type_New,Sale Type_Oth,Sale Type_VWD,Sale Type_WD,Sale Condition_AdjLand,Sale Condition_Alloca,Sale Condition_Family,Sale Condition_Normal,Sale Condition_Partial
0,141.0,31770,6,5,1960,1960,112.0,639.0,0.0,441.0,...,0,0,0,0,1,0,0,0,1,0
1,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,270.0,...,0,0,0,0,1,0,0,0,1,0
2,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,406.0,...,0,0,0,0,1,0,0,0,1,0
3,93.0,11160,7,5,1968,1968,0.0,1065.0,0.0,1045.0,...,0,0,0,0,1,0,0,0,1,0
4,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,137.0,...,0,0,0,0,1,0,0,0,1,0
5,78.0,9978,6,6,1998,1998,20.0,602.0,0.0,324.0,...,0,0,0,0,1,0,0,0,1,0
6,41.0,4920,8,5,2001,2001,0.0,616.0,0.0,722.0,...,0,0,0,0,1,0,0,0,1,0
7,43.0,5005,8,5,1992,1992,0.0,263.0,0.0,1017.0,...,0,0,0,0,1,0,0,0,1,0
8,39.0,5389,8,5,1995,1996,0.0,1180.0,0.0,415.0,...,0,0,0,0,1,0,0,0,1,0
9,60.0,7500,7,5,1999,1999,0.0,0.0,0.0,994.0,...,0,0,0,0,1,0,0,0,1,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2925 entries, 0 to 2924
Columns: 274 entries, Lot Frontage to Sale Condition_Partial
dtypes: float64(11), int64(263)
memory usage: 6.1 MB


**ЗАДАНИЕ: Мы будем пытаться спрогнозировать значение колонки SalePrice. Разделите данные на две части - признаки X и целевая переменная y.**

In [10]:
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

**ЗАДАНИЕ: С помощью scikit-learn разделите и X, и y на обучающий и тестовый наборы данных. Поскольку далее мы будем использовать поиск по сетке, то выделите под тестовые данные 10% от всех данных. Чтобы получить такое же разбиение данных, как и в нашем блокноте, можете использовать random_state = 101.**

In [11]:
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=101)

**ЗАДАНИЕ: Признаки в этом наборе данных имеют различный масштаб и различные единицы измерения. Для оптимальной работы регрессии, выполните масштабирование признаков X. Обратите внимание на то, какие данные подавать на вход для .fit(), а какие данные на вход для .transform().**

In [17]:
from sklearn.preprocessing import StandardScaler

In [18]:
scaler = StandardScaler()

In [19]:
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

**ЗАДАНИЕ: Мы будем использовать модель "Elastic Net". Создайте экземпляр модели ElasticNet в scikit-learn, используя параметры по умолчанию.**

In [20]:
from sklearn.linear_model import ElasticNet

In [21]:
base_elastic_net_model = ElasticNet()

**ЗАДАНИЕ: Модель Elastic Net работает с двумя основными параметрами - alpha и L1_ratio. Создайте словарь с набором различных значений этих параметров, для поиска по сетке. Вы можете выбрать различные значения параметров, но имейте ввиду, что Ваши результаты могут не совпасть с результатами в блокноте с решениями.**

In [26]:
param_grid = {'alpha': [1, 5, 25, 45, 55, 75, 95, 100], 'l1_ratio': [.1, .2, .3, .5, .7, .9, .95, .98, 1]}

**ЗАДАНИЕ: С помощью scikit-learn создайте объект GridSearchCV и запустите поиск по сетке для нахождения наилучших параметров модели, используя обучающие данные (предварительно смасштабированные). [Для некоторых комбинаций параметров Вы можете получить предупреждения (warnings).](https://stackoverflow.com/questions/20681864/lasso-on-sklearn-does-not-converge)**

In [27]:
from sklearn.model_selection import GridSearchCV

In [36]:
grid_model = GridSearchCV(estimator=base_elastic_net_model,
                          param_grid=param_grid, cv=10,
                          scoring='neg_mean_squared_error',
                          verbose=2)
grid_model.fit(X_train, y_train)

Fitting 10 folds for each of 72 candidates, totalling 720 fits
[CV] END ..............................alpha=1, l1_ratio=0.1; total time=   0.0s
[CV] END ..............................alpha=1, l1_ratio=0.1; total time=   0.1s
[CV] END ..............................alpha=1, l1_ratio=0.1; total time=   0.1s
[CV] END ..............................alpha=1, l1_ratio=0.1; total time=   0.2s
[CV] END ..............................alpha=1, l1_ratio=0.1; total time=   0.1s
[CV] END ..............................alpha=1, l1_ratio=0.1; total time=   0.1s
[CV] END ..............................alpha=1, l1_ratio=0.1; total time=   0.1s
[CV] END ..............................alpha=1, l1_ratio=0.1; total time=   0.1s
[CV] END ..............................alpha=1, l1_ratio=0.1; total time=   0.1s
[CV] END ..............................alpha=1, l1_ratio=0.1; total time=   0.1s
[CV] END ..............................alpha=1, l1_ratio=0.2; total time=   0.1s
[CV] END ..............................alpha=1

/home/rostya/.local/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.366e+11, tolerance: 1.537e+09
  model = cd_fast.enet_coordinate_descent(


[CV] END ................................alpha=1, l1_ratio=1; total time=   0.3s


/home/rostya/.local/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.572e+11, tolerance: 1.534e+09
  model = cd_fast.enet_coordinate_descent(


[CV] END ................................alpha=1, l1_ratio=1; total time=   0.3s


...

/home/rostya/.local/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.838e+09, tolerance: 1.567e+09
  model = cd_fast.enet_coordinate_descent(


[CV] END ...............................alpha=45, l1_ratio=1; total time=   0.3s
[CV] END ...............................alpha=45, l1_ratio=1; total time=   0.3s
[CV] END ...............................alpha=45, l1_ratio=1; total time=   0.3s
[CV] END ...............................alpha=45, l1_ratio=1; total time=   0.3s
[CV] END ...............................alpha=45, l1_ratio=1; total time=   0.3s


/home/rostya/.local/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.593e+09, tolerance: 1.546e+09
  model = cd_fast.enet_coordinate_descent(


[CV] END ...............................alpha=45, l1_ratio=1; total time=   0.2s
[CV] END .............................alpha=55, l1_ratio=0.1; total time=   0.1s
[CV] END .............................alpha=55, l1_ratio=0.1; total time=   0.1s
[CV] END .............................alpha=55, l1_ratio=0.1; total time=   0.1s
[CV] END .............................alpha=55, l1_ratio=0.1; total time=   0.1s
[CV] END .............................alpha=55, l1_ratio=0.1; total time=   0.0s
[CV] END .............................alpha=55, l1_ratio=0.1; total time=   0.1s
[CV] END .............................alpha=55, l1_ratio=0.1; total time=   0.1s
[CV] END .............................alpha=55, l1_ratio=0.1; total time=   0.0s
[CV] END .............................alpha=55, l1_ratio=0.1; total time=   0.1s
[CV] END .............................alpha=55, l1_ratio=0.1; total time=   0.1s
[CV] END .............................alpha=55, l1_ratio=0.2; total time=   0.0s
[CV] END ...................

GridSearchCV(cv=10, estimator=ElasticNet(),
             param_grid={'alpha': [1, 5, 25, 45, 55, 75, 95, 100],
                         'l1_ratio': [0.1, 0.2, 0.3, 0.5, 0.7, 0.9, 0.95, 0.98,
                                      1]},
             scoring='neg_mean_squared_error', verbose=2)

**ЗАДАНИЕ: Отобразите наилучшую комбинацию параметров для Вашей модели.**

In [38]:
grid_model.best_params_

{'alpha': 100, 'l1_ratio': 1}

**ЗАДАНИЕ: Оцените работу модели на тестовом наборе данных (предварительно смасштабированном) в 10%, которые модель ещё не видела. В блокноте с решениями мы получили MAE = $\$$14149 и RMSE = $\$$20532**

In [43]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [40]:
y_pred = grid_model.predict(X_test)

In [42]:
MAE = mean_absolute_error(y_test, y_pred)
MAE

14195.35490056217

In [44]:
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
RMSE

20558.50856689317

In [45]:
np.mean(df['SalePrice'])

180815.53743589742

## Отличная работа!

----